In [64]:
import numpy as np
import pandas as pd
import datetime
import os

# 1. Dataset conversion

In [65]:
dynamic = pd.read_csv('./sph_dynamic.csv')
static = pd.read_csv('./sph_static.csv')

Merge the `dynamic` and `static` dataset based on the `stay_id`

In [66]:
merge_2 = pd.merge(dynamic, static, on = 'stay_id')

Formatting columns and vent_start datetime format

In [67]:
merge_2.drop(['vent_end'], axis = 1, inplace=True)

merge_2['charttime'] = pd.to_datetime(merge_2['charttime'], format = "%Y-%m-%d %H:%M:%S.%f")
merge_2['icu_intime'] = pd.to_datetime(merge_2['icu_intime'], format = "%Y-%m-%d %H:%M:%S.%f")
for i in range(len(merge_2)):
        if pd.Timestamp(merge_2.loc[i, 'vent_start']).replace(year = merge_2.loc[i, 'icu_intime'].year) - merge_2.loc[i, 'icu_intime'] < datetime.timedelta(seconds=0):
            merge_2.loc[i, 'vent_start'] = pd.to_datetime(merge_2.loc[i, 'vent_start'], format='%m/%d/%y %H:%M')
            i_datetime = merge_2.loc[i, 'icu_intime']
            i_year = i_datetime.year
            i_month = i_datetime.month
            i_day = i_datetime.day
            i_hour = i_datetime.hour
            i_minute = i_datetime.minute
            i_second = i_datetime.second
            merge_2.loc[i, 'vent_start'] = merge_2.loc[i, 'vent_start'].replace(year = i_year, 
                                                                                   month = i_month, 
                                                                                   day = i_day, 
                                                                                   hour = i_hour, 
                                                                                   minute = i_minute, 
                                                                                   second = i_second)   
        else:
            merge_2.loc[i, 'vent_start'] = pd.Timestamp(merge_2.loc[i, 'vent_start']).replace(year = merge_2.loc[i, 'icu_intime'].year)
merge_2['vent_start'] = pd.to_datetime(merge_2['vent_start'], format = "%Y-%m-%d %H:%M:%S.%f")



Drop > patients intubated outside +48h icu_intime

In [68]:
# Remove the chart data that is done after 12h in ICU
#merge_2 = merge_2[(merge_2['charttime'] - merge_2['icu_intime']).dt.total_seconds() <= 43200]  

# Remove the records of the patient that is intubated after 48h in ICU
merge_2 = merge_2[(merge_2['vent_start'] - merge_2['icu_intime']).dt.total_seconds() <= 172800]

# Specific case: Last year of the day
#merge_2 = merge_2[(merge_2['vent_start'] - merge_2['icu_intime']).dt.total_seconds() >= 0]

Comvert timestamps to timedelta

In [69]:
merge_2['charttime'] = pd.to_datetime(merge_2['charttime']) - pd.to_datetime(merge_2['icu_intime'])
merge_2['charttime'] = merge_2['charttime'].dt.total_seconds() / 3600

merge_2['vent_start'] = pd.to_datetime(merge_2['vent_start']) - pd.to_datetime(merge_2['icu_intime'])
merge_2['vent_start'] = merge_2['vent_start'].dt.total_seconds() / 3600

merge_2['icu_intime'] = pd.to_datetime(merge_2['icu_intime']) - pd.to_datetime(merge_2['icu_intime'])
merge_2['icu_intime'] = merge_2['icu_intime'].dt.total_seconds() / 3600

In [70]:
merge_2 = merge_2.sort_values(['stay_id', 'charttime']) 
merge_2


,stay_id,charttime,total_protein,calcium,creatinine,glucose,sodium,chloride,heart_rate,sbp,...,urineoutput,sofa_respiration,sofa_coagulation,sofa_liver,sofa_cardiovascular,sofa_cns,sofa_renal,icu_intime,vent_start,vent_duration
6204,30004144,-3.840278,NaN,8.0,0.7,114.0,135.0,103.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,26.659722,25.000000
6203,30004144,-0.240278,NaN,6.8,0.7,133.0,135.0,102.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,26.659722,25.000000
6991,30005366,8.550278,NaN,8.8,6.7,41.0,139.0,100.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20.383611,6.000000
2701,30006983,-14.611667,NaN,8.0,0.8,156.0,132.0,102.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.055000,49.000000
2700,30006983,0.521667,NaN,7.1,1.0,89.0,136.0,108.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.055000,49.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3896,39985110,6.653889,NaN,10.4,6.8,149.0,139.0,98.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38.770556,31.266667
5933,39986206,0.508056,NaN,7.6,5.6,121.0,139.0,103.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,22.574722,222.000000
5932,39986206,6.924722,NaN,7.5,6.0,101.0,139.0,103.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,22.574722,222.000000
4182,39992167,2.200000,NaN,7.9,0.9,113.0,131.0,100.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,22.000000,84.000000


In [71]:
merge_2.to_csv('merge2.csv', index=False)

# 2. Drop high missing

In [72]:
missing_lst = []

for column in merge_2.columns:
    missing_lst.append(( column, (merge_2[column].isna().sum() / merge_2.shape[0]) ))

missing_sorted = sorted(missing_lst, key = lambda lst: lst[1])

print()
print('Missing rate of each column:')
print()
for i in missing_sorted:
    print(i[0] + ' ' + '{:.2%}'.format(i[1]))

print()
print('Dropped columns:')
print()

dropped_merge = merge_2
for i in missing_sorted:
    if i[1] > 0.4:
        dropped_merge = dropped_merge.drop(i[0], axis = 1)
        print('Drop', i[0])

        
dropped_merge_missing_lst = []   
print()
print('Remaining columns:')
print()
for i in dropped_merge:
    for j in missing_sorted:
        if i in j[0]:
            dropped_merge_missing_lst.append((j[0], j[1]))

dropped_merge_missing_lst = sorted(dropped_merge_missing_lst, key = lambda lst: lst[1])
for i in dropped_merge_missing_lst:
    print(i[0] + ' ' + '{:.2%}'.format(i[1]))
    
dropped_merge.columns


Missing rate of each column:

stay_id 0.00%
charttime 0.00%
icu_intime 0.00%
vent_start 0.00%
vent_duration 0.00%
sodium 3.01%
chloride 3.32%
creatinine 3.47%
glucose 5.81%
calcium 12.99%
hemoglobin 16.41%
wbc 16.75%
pt 43.62%
ast 56.61%
alt 56.98%
bilirubin_total 56.98%
alp 57.19%
bilirubin_direct 96.91%
bilirubin_indirect 96.96%
resp_rate 97.15%
heart_rate 97.17%
sofa_cardiovascular 97.77%
mbp 97.93%
sbp 98.05%
dbp 98.05%
total_protein 98.60%
urineoutput 98.81%
temperature 99.27%
sofa_cns 99.30%
ph 99.69%
sofa_respiration 99.77%
lactate 99.82%
sofa_coagulation 99.98%
sofa_liver 99.98%
sofa_renal 100.00%

Dropped columns:

Drop pt
Drop ast
Drop alt
Drop bilirubin_total
Drop alp
Drop bilirubin_direct
Drop bilirubin_indirect
Drop resp_rate
Drop heart_rate
Drop sofa_cardiovascular
Drop mbp
Drop sbp
Drop dbp
Drop total_protein
Drop urineoutput
Drop temperature
Drop sofa_cns
Drop ph
Drop sofa_respiration
Drop lactate
Drop sofa_coagulation
Drop sofa_liver
Drop sofa_renal

Remaining columns

Index(['stay_id', 'charttime', 'calcium', 'creatinine', 'glucose', 'sodium',
       'chloride', 'hemoglobin', 'wbc', 'icu_intime', 'vent_start',
       'vent_duration'],
      dtype='object')

# 3.A keep vent_duration 90th percentile

In [84]:
'{:.4f}'.format(dropped_merge['vent_duration'].quantile(0.9))

'104.1333'

In [85]:
working_vent_dur_90_all = dropped_merge.loc[dropped_merge['vent_duration'] < 104.1333]
working_vent_dur_90_all.to_csv('working_vent_dur_90_all.csv', index = False)

In [ ]:
from ydata_profiling import ProfileReport

profile_working_vent_dur_90_all = ProfileReport(working_vent_dur_90_all, title="working_vent_dur_90_all Profiling Report")

profile_working_vent_dur_90_all.to_notebook_iframe()


In [ ]:
profile_working_vent_dur_90_all.to_file('working_vent_dur_90_all.html')


## 3.A.1 Train:Test split then Median Impute then  Min Max Scaling

In [88]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# Median impute function based on scikit package
def imputeMedian(dataframe):
    df = dataframe.values
    imp_median = SimpleImputer(missing_values = np.nan, strategy='median')
    df = imp_median.fit_transform(df)
    # put back the column label
    df = pd.DataFrame(df, columns = list(dataframe.columns))
    return df

# Min Max scaling function based on scikit package
def minMaxScale(dataframe, lst):
    scaler = MinMaxScaler()
    df = dataframe.copy(deep=True)
    for i in lst:
        df[i] = scaler.fit_transform(df[i].values.reshape(-1, 1))
    return pd.DataFrame(df)

In [89]:
# Get all unique values of column A as a list
unique_values = working_vent_dur_90_all['stay_id'].unique().tolist()

# Set the random seed for reproducibility
np.random.seed(42)

# Randomly split 70% of the unique values into a train list and the remaining 30% into a test list
train_size = int(0.7 * len(unique_values))
train = np.random.choice(unique_values, size=train_size, replace=False).tolist()
test = [value for value in unique_values if value not in train]

# Split the DataFrame into train and test DataFrames based on the values of stay_id
train_90_df = working_vent_dur_90_all[working_vent_dur_90_all['stay_id'].isin(train)].reset_index(drop = True).sort_values(['stay_id', 'charttime'])
test_90_df = working_vent_dur_90_all[working_vent_dur_90_all['stay_id'].isin(test)].reset_index(drop = True).sort_values(['stay_id', 'charttime'])
X_test = test_90_df.loc[:, ~test_90_df.columns.isin(['stay_id','icu_intime', 'vent_duration'])]


# Median impute for train. Median impute for test
train_90_df = imputeMedian(train_90_df)
test_90_df = imputeMedian(test_90_df)


# Min Max scaling for train. Min Max scaling for test


train_90_df_scale = minMaxScale(train_90_df, ['charttime', 'calcium', 
                                              'creatinine', 'glucose', 
                                              'sodium','chloride', 
                                              'hemoglobin', 'wbc', 
                                              'icu_intime', 'vent_start',
                                              'vent_duration'])

test_90_df_scale = minMaxScale(test_90_df,  ['charttime', 'calcium', 
                                              'creatinine', 'glucose', 
                                              'sodium','chloride', 
                                              'hemoglobin', 'wbc', 
                                              'icu_intime', 'vent_start',
                                              'vent_duration'])

print('Train_90 DataFrame:')
print(train_90_df)
print()
print('Train_90 Min Max scaled DataFrame:')
print(train_90_df_scale)
print()

print('Test_90 DataFrame:')
print(test_90_df)
print()
print('Test_90 Min Max scaled DataFrame:')
print(test_90_df_scale)

Train_90 DataFrame:
         stay_id  charttime  calcium  creatinine  glucose  sodium  chloride  \
0     30004144.0  -3.840278      8.0         0.7    114.0   135.0     103.0   
1     30004144.0  -0.240278      6.8         0.7    133.0   135.0     102.0   
2     30006983.0 -14.611667      8.0         0.8    156.0   132.0     102.0   
3     30006983.0   0.521667      7.1         1.0     89.0   136.0     108.0   
4     30023204.0   0.518056      8.2         1.6    124.0   130.0     100.0   
...          ...        ...      ...         ...      ...     ...       ...   
3618  39972274.0  -8.366667      9.3         0.9     89.0   140.0     106.0   
3619  39972274.0   4.600000      9.0         0.7     74.0   140.0     108.0   
3620  39985110.0 -11.296111     10.2         5.2    125.0   138.0      98.0   
3621  39985110.0  -8.379444     10.1         5.6    126.0   138.0      98.0   
3622  39985110.0   6.653889     10.4         6.8    149.0   139.0      98.0   

      hemoglobin   wbc  icu_int

In [90]:
train_90_df.to_csv('train_90_df.csv')
train_90_df_scale.to_csv('train_90_df_scale.csv')
test_90_df.to_csv('test_90_df.csv')
test_90_df_scale.to_csv('test_90_df_scaled.csv')

In [ ]:
from ydata_profiling import ProfileReport

profile_train_90_df = ProfileReport(train_90_df, title="train_90_df Profiling Report")
profile_train_90_df.to_notebook_iframe()

In [ ]:
profile_train_90_df.to_file('profile_train_90_df.html')

In [ ]:
profile_train_90_df_scale = ProfileReport(train_90_df_scale, title="train_90_df_scale Profiling Report")
profile_train_90_df_scale.to_notebook_iframe()

In [ ]:
profile_train_90_df_scale.to_file('profile_train_90_df_scale.html')

In [ ]:
profile_test_90_df = ProfileReport(test_90_df, title="test_90_df Profiling Report")
profile_test_90_df.to_notebook_iframe()

In [ ]:
profile_test_90_df.to_file('profile_test_90_df.html')

In [ ]:
profile_test_90_df_scale = ProfileReport(test_90_df_scale, title="test_90_df_scale Profiling Report")
profile_test_90_df_scale.to_notebook_iframe()

In [ ]:
profile_test_90_df_scale.to_file('profile_test_90_df_scale.html')

# 3.B vent_duration 100th percentile

## 3.B.1 Train:Test split then Median Impute then  Min Max Scaling

In [73]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# Median impute function based on scikit package
def imputeMedian(dataframe):
    df = dataframe.values
    imp_median = SimpleImputer(missing_values = np.nan, strategy='median')
    df = imp_median.fit_transform(df)
    # put back the column label
    df = pd.DataFrame(df, columns = list(dataframe.columns))
    return df

# Min Max scaling function based on scikit package
def minMaxScale(dataframe, lst):
    scaler = MinMaxScaler()
    df = dataframe.copy(deep=True)
    for i in lst:
        df[i] = scaler.fit_transform(df[i].values.reshape(-1, 1))
    return pd.DataFrame(df)

In [74]:
# Get all unique values of column A as a list
unique_values = dropped_merge['stay_id'].unique().tolist()

# Set the random seed for reproducibility
np.random.seed(42)

# Randomly split 70% of the unique values into a train list and the remaining 30% into a test list
train_size = int(0.7 * len(unique_values))
train = np.random.choice(unique_values, size=train_size, replace=False).tolist()
test = [value for value in unique_values if value not in train]

# Split the DataFrame into train and test DataFrames based on the values of stay_id
train_100_df = dropped_merge[dropped_merge['stay_id'].isin(train)].reset_index(drop = True).sort_values(['stay_id', 'charttime'])
test_100_df = dropped_merge[dropped_merge['stay_id'].isin(test)].reset_index(drop = True).sort_values(['stay_id', 'charttime'])
X_test = test_100_df.loc[:, ~test_100_df.columns.isin(['stay_id','icu_intime', 'vent_duration'])]


# Median impute for train. Median impute for test
train_100_df = imputeMedian(train_100_df)
test_100_df = imputeMedian(test_100_df)


# Min Max scaling for train. Min Max scaling for test


train_100_df_scale = minMaxScale(train_100_df, ['charttime', 'calcium', 
                                              'creatinine', 'glucose', 
                                              'sodium','chloride', 
                                              'hemoglobin', 'wbc', 
                                              'icu_intime', 'vent_start',
                                              'vent_duration'])

test_100_df_scale = minMaxScale(test_100_df,  ['charttime', 'calcium', 
                                              'creatinine', 'glucose', 
                                              'sodium','chloride', 
                                              'hemoglobin', 'wbc', 
                                              'icu_intime', 'vent_start',
                                              'vent_duration'])

print('Train_100 DataFrame:')
print(train_100_df)
print()
print('Train_100 Min Max scaled DataFrame:')
print(train_100_df_scale)
print()

print('Test_100 DataFrame:')
print(test_100_df)
print()
print('Test_100 Min Max scaled DataFrame:')
print(test_100_df_scale)

Train_100 DataFrame:
         stay_id  charttime  calcium  creatinine  glucose  sodium  chloride  \
0     30004144.0  -3.840278      8.0         0.7    114.0   135.0     103.0   
1     30004144.0  -0.240278      6.8         0.7    133.0   135.0     102.0   
2     30006983.0 -14.611667      8.0         0.8    156.0   132.0     102.0   
3     30006983.0   0.521667      7.1         1.0     89.0   136.0     108.0   
4     30023204.0   0.518056      8.2         1.6    124.0   130.0     100.0   
...          ...        ...      ...         ...      ...     ...       ...   
4394  39972274.0   4.600000      9.0         0.7     74.0   140.0     108.0   
4395  39977971.0 -10.712500      9.0         1.9     96.0   134.0      98.0   
4396  39977971.0  -5.129167      8.4         1.1    123.0   138.0     102.0   
4397  39977971.0   1.220833      8.5         2.1    202.0   129.0      96.0   
4398  39977971.0   9.537500      8.9         2.2     98.0   132.0      97.0   

      hemoglobin   wbc  icu_in

In [75]:
train_100_df.to_csv('train_100_df.csv')
train_100_df_scale.to_csv('train_100_df_scale.csv')
test_100_df.to_csv('test_100_df.csv')
test_100_df_scale.to_csv('test_100_df_scaled.csv')

In [ ]:
from ydata_profiling import ProfileReport

profile_train_100_df = ProfileReport(train_100_df, title="train_100_df Profiling Report")
profile_train_100_df.to_notebook_iframe()

In [ ]:
profile_train_100_df.to_file('profile_train_100_df.html')

In [ ]:
profile_train_100_df_scale = ProfileReport(train_100_df_scale, title="train_100_df_scale Profiling Report")
profile_train_100_df_scale.to_notebook_iframe()

In [ ]:
profile_train_100_df_scale.to_file('profile_train_100_df_scale.html')

In [ ]:
profile_test_100_df = ProfileReport(test_100_df, title="test_100_df Profiling Report")
profile_test_100_df.to_notebook_iframe()

In [ ]:
profile_test_100_df.to_file('profile_test_100_df.html')

In [ ]:
profile_test_100_df_scale = ProfileReport(test_100_df_scale, title="test_100_df_scale Profiling Report")
profile_test_100_df_scale.to_notebook_iframe()

In [ ]:
profile_test_100_df_scale.to_file('profile_test_100_df_scale.html')